In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classificação de textos com Generative AI na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Modelos generativos como o PaLM 2 são modelos de linguagem poderosos usados para várias tarefas de processamento de linguagem natural (NLP). Uma delas é a classificação de texto, que envolve a atribuição de uma ou mais categorias a um determinado trecho de texto. Embora a classificação de texto possa ser feita usando técnicas NLP tradicionais, os LLMs podem realizar a classificação fornecendo prompts (em oposição a dados rotulados específicos do domínio), o que pode acelerar o tempo necessário para criar uma solução de classificação de texto. Os modelos de classificação baseados em LLMs podem ser ajustados com muitos exemplos por meio de treinamento de modelo personalizado, mas isso está além do escopo deste notebook.

Neste notebook, você explorará como fazer classificação de texto usando prompts com a API PaLM. Saiba mais sobre os prompts de classificação na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/classification-prompts).

### Objetivo

No final do notebook, você será capaz de usar um LLM para executar várias tarefas de classificação, incluindo:

* Classificação de texto com prompt zero-shot
* Classificação de texto com prompt few-shot
* Tarefas como:
     * Análise de sentimentos
     * Classificação do tópico
     * Detecção de spam
     * Reconhecimento de intenção
     * Identificação do idioma
     * Detecção de toxicidade
     * Detecção de emoções

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)


import pandas as pd
from vertexai.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Classificação de texto

Na seção abaixo, você explorará prompts zero-shot, few-shot e alguns tipos comuns de tarefas de classificação de texto.

### Prompt zero-shot

O prompt zero-shot é onde você não fornece exemplos com rótulos e conta com o LLM para fazer a classificação por conta própria.

In [ ]:
prompt = """
Classifique a texto abaixo:\n
texto: "Hoje eu vi um animal peludo no parque - Ele tinha um rabo logo e grandes olhos."
classe: cachorros, gatos
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

### Prompt few-shot

Com prompt few-shot, você fornece exemplos para o modelo PaLM e espera que ele faça a classificação com base nos exemplos fornecidos.

In [ ]:
prompt = """
Qual é o assunto de uma determinada manchete de notícias? \n
- negócios \n
- entretenimento \n
- saúde \n
- esportes \n
- tecnologia \n\n

Texto: revisão prática do especialista do Pixel 7 Pro. \n
A resposta é: tecnologia \n

Texto: Parar de fumar? \n
A resposta é: saúde \n

Texto: Passarinhos ou bichos-papões? As 5 principais dicas para bater abaixo do par \n
A resposta é: esportes \n

Texto: Alívio do aumento do salário mínimo local parece mais remoto \n
A resposta é: negócios \n

Texto: Você não vai adivinhar quem acabou de chegar a Bari, na Itália, para a estreia do filme. \n
A resposta é:
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

### Outros exemplos de classificação

Explore alguns prompts de classificação de texto mais comuns abaixo, todos baseados em prompts zero-shot. Você também pode transformar alguns deles em prompts few-shot, fornecendo seus próprios exemplos personalizados de texto e as classes de saída associadas.

#### Classificação de tópico

In [ ]:
prompt = """
Classifique um trecho de texto em um dos vários tópicos predefinidos, como esportes, política ou entretenimento. \n
texto: O presidente Biden visitará a Índia no mês de março para discutir algumas oportunidades. \n
classe:
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

####  Detecção de spam

In [ ]:
prompt = """
Dado um e-mail, classifique-o como spam ou não spam. \n
e-mail: oi usuário, \n
       você foi selecionado como vencedor da loteria e pode ganhar até 1 milhão de dólares. \n
       gentilmente compartilhe seus dados bancários e podemos prosseguir a partir daí. \n\n

       de, \n
       Departamento oficial de loteria dos EUA
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

#### Reconhecimento de intenção

In [ ]:
prompt = """
Dada a entrada de um usuário, classifique sua intenção, como "encontrar informações", "fazer uma reserva" ou "fazer um pedido". \n
entrada do usuário: Olá, você pode agendar uma mesa para dois no Restaurante Google para 1º de maio?
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

#### Identificação de linguagem

In [ ]:
prompt = """
Dado um pedaço de texto, classifique o idioma em que está escrito. \n
texto: Selam nasıl gidiyor?
linguagem:
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

#### Detecção de toxicidade

In [ ]:
prompt = """
Dado um trecho de texto, classifique-o como tóxico ou não tóxico. \n
texto: Tente que você consegue.
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.1).text
)

#### Detecção de emoção

In [ ]:
prompt = """
Dado um trecho de texto, classifique a emoção que ele transmite, como 'felicidade' ou 'raiva'. \n
texto: Estou muito feliz com as notícias de ontem
"""

print(
    generation_model.predict(prompt=prompt, max_output_tokens=256, temperature=0.5).text
)

### Avaliação

Você pode avaliar as saídas da tarefa de classificação de texto se as classes de informações básicas estiverem disponíveis. Para mostrar como isso funciona, comece criando um dataframe simples com análises de produtos e o sentimento de verdade.

In [ ]:
review_data = {
    "review": [
        "adoro este produto. Tem tudo o que procuro!",
        "tudo o que posso dizer é que você ficará feliz depois de comprar este produto",
        "é muito caro e não vale o preço",
        "Ele é ok. Não é bom nem muito ruim.",
    ],
    "sentiment_groundtruth": ["positivo", "positivo", "negativo", "neutro"],
}

review_data_df = pd.DataFrame(review_data)
review_data_df

Agora que você tem os dados com avaliações e sentimentos como rótulos de informações básicas, pode chamar o modelo de geração de texto para cada linha de avaliação usando a função `apply`. Cada linha usará o prompt na coluna `review` para prever o sentimento usando a API PaLM e armazenará os resultados na coluna `sentiment_prediction`.

In [ ]:
def get_sentiment(row):
    prompt = f"""Classifique o sentimento dos reviews abaixo como "positivo", "neutro" ou "negativo". \n\n
                review: {row} \n
                sentimento:
              """
    response = generation_model.predict(prompt=prompt).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

No final, você pode chamar a função `classification_report` do móulo `scikit-learn` para medir a precisão e outras métricas de classificação, passando sentimentos de verdade absoluta `sentiment_groundtruth` e sentimento previsto `sentiment_prediction`:

In [ ]:
from sklearn.metrics import classification_report

print(
    classification_report(
        review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
    )
)